import libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re

read in data

In [9]:
nps = pd.read_csv("Georgia_NPs_AddressesNPIs_new.csv")

In [10]:

KNOWN_CREDS = [
    "RN","LPN","LVN","APRN","NP","FNP","PMHNP","AGNP","PNP","WHNP",
    "CRNA","CNM","CNS","CNL",
    "ADN","BSN","MSN","DNP","PHD","MPH","MS","MA","MBA",
    "MD","DO","PA","PA-C",
]
KNOWN_CREDS_SORTED = sorted(KNOWN_CREDS, key=len, reverse=True)
KNOWN_RE = re.compile(r"(?:%s)" % "|".join(map(re.escape, KNOWN_CREDS_SORTED)))

SYNONYMS = {
    "PH.D": "PHD", "PH D": "PHD", "P H D": "PHD",
    "D.N.P": "DNP", "M.S.N": "MSN", "B.S.N": "BSN", "R.N": "RN", "N.P": "NP",
    "phd": "PHD", "dnp": "DNP", "msn": "MSN", "bsn": "BSN", "rn": "RN", "np": "NP",
    "fnp": "FNP", "pmhnp": "PMHNP", "agnp": "AGNP", "pnp": "PNP", "whnp": "WHNP",
    "aprn": "APRN", "crna": "CRNA", "cnm": "CNM", "cns": "CNS", "cnl": "CNL",
    "md": "MD", "do": "DO", "pa": "PA", "pa-c": "PA-C",
}

def normalize_token(token: str) -> str:
    t = token.strip()
    if not t:
        return ""
    t = t.replace(".", "")
    t = re.sub(r"\s+", " ", t).strip()
    key = t.lower()
    if key in SYNONYMS:
        return SYNONYMS[key]
    return t.upper()

def split_stuck_credentials(token: str) -> list[str]:
    s = normalize_token(token)
    if not s:
        return []
    if s in KNOWN_CREDS:
        return [s]
    out, i = [], 0
    while i < len(s):
        m = KNOWN_RE.match(s, i)
        if not m:
            return [s]  # keep as-is if it can't be decomposed
        out.append(m.group(0))
        i = m.end()
    return out

def clean_credentials_cell(x):
    if pd.isna(x) or not isinstance(x, str) or x.strip() == "":
        return x  # leave blanks/NaNs untouched

    s = x.strip()

    # normalize separators -> commas
    s = re.sub(r"[;\/|&]+", ",", s)
    s = re.sub(r"\band\b", ",", s, flags=re.IGNORECASE)

    # spaced hyphens between words become commas
    s = re.sub(r"(?<=\w)\s*-\s*(?=\w)", ",", s)

    # remove dangling leading/trailing hyphens  (FIXED REGEX)
    s = re.sub(r"^-+|-+$", "", s)

    # remove periods globally
    s = s.replace(".", "")

    # collapse whitespace and tidy commas
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\s*,\s*", ",", s)
    s = re.sub(r",+", ",", s).strip(",")

    if not s:
        return ""

    rough_tokens = [t for t in s.split(",") if t.strip()]

    exploded = []
    for t in rough_tokens:
        exploded.extend(split_stuck_credentials(t))

    normalized = [normalize_token(t) for t in exploded if t]
    unique = sorted(set([t for t in normalized if t]))

    return ",".join(unique)

# Apply to your column
nps['Provider_Credentials'] = nps['Provider_Credentials'].apply(clean_credentials_cell)

# sort rows by the cleaned column
nps = nps.sort_values(by='Provider_Credentials', ascending=True).reset_index(drop=True)

display(nps)

,NPI,Entity_Type_Code,Provider_Last_Name,Provider_First_Name,Provider_Credentials,Street1,Street2,City,State,ZIP,...,Provider_Enumeration_Date,Provider_Sex,Provider_Taxonomy_Code,Provider_License_Number_1,Healthcare_Provider_Primary_Tax,Certification_Date,MEDICARE_SPECIALTY_CODE,MEDICARE_PROVIDER_SUPPLIER_TYPE,PROVIDER_TAXONOMY_DESCRIPTION_,NP_Type
0,1932582301,1,JIMENEZ,SARAH,"A,APN,C",1120 15TH ST,NaN,AUGUSTA,GA,30912,...,7/3/2015,F,363L00000X,26NJ00573000,N,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,"NP, No Subspecialty Noted"
1,1720508567,1,FLOYD,CHRISTOPH,"A,C,FNP",1641 MADISON AVE,NaN,TIFTON,GA,317943757,...,6/22/2017,M,363LF0000X,213107,Y,3/23/2021,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Family NP
2,1659744316,1,WAGNER,KELLY,"A,C,FNP",1395 EISENHOWER DR,NaN,SAVANNAH,GA,314063901,...,11/3/2015,F,363LF0000X,RN266801,Y,6/12/2023,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Family NP
3,1851955819,1,LAPOINTE,JODEAN,"A,C,FNP",2712 LAWRENCEVILLE HWY,NaN,DECATUR,GA,300332512,...,4/25/2019,F,363L00000X,RN150178,Y,8/19/2020,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,"NP, No Subspecialty Noted"
4,1548556426,1,MILEY,BARBARA,"A,C,FNP",3000 HOSPITAL BLVD,NaN,ROSWELL,GA,300764915,...,6/20/2011,F,363LA2100X,RN165608,N,3/28/2025,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14462,1902220429,1,PETITO,ANDI,NaN,3110 CLIFTON SPRINGS RD,NaN,DECATUR,GA,300344600,...,2/5/2014,F,363LW0102X,RN225020,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14463,1932484201,1,WATERS,CATHLEEN,NaN,604 RIVERSIDE AVE,NaN,WAYCROSS,GA,315015323,...,10/18/2011,F,363LW0102X,RN167697,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14464,1952026031,1,CENTER,SYDNEY,NaN,1595 KENNESAW DUE WEST RD NW STE 100,NaN,KENNESAW,GA,301527640,...,10/11/2022,F,363LW0102X,RN309563,Y,12/7/2023,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14465,1952876369,1,WELLS,JESSICA,NaN,477 WINDSOR ST SW,NaN,ATLANTA,GA,303122530,...,10/11/2018,F,363LW0102X,RN185551,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP


In [11]:
# Create a frequency table of Provider_Credentials
credential_counts = (
    nps['Provider_Credentials']
    .value_counts()
    .rename_axis('Provider_Credentials')
    .reset_index(name='Count')
    .sort_values(by='Provider_Credentials', ascending=True)
    .reset_index(drop=True)
)

# Display the summary table
display(credential_counts)

,Provider_Credentials,Count
0,"A,APN,C",1
1,"A,C,FNP",6
2,"A,C,GNP",7
3,"A,CPNP",1
4,"A,G,MSN",1
...,...,...
458,SNP,1
459,WHCNP,2
460,WHNP,65
461,WHPN,1


In [12]:
mask_unspecified = nps["NP_Type"].str.contains("no subspecialty", case=False, na=False)
nps.loc[mask_unspecified, "NP_Type"].value_counts()

nps.loc[mask_unspecified, "Provider_Credentials"].value_counts()

Provider_Credentials
NP          837
C,NP        318
C,FNP       202
FNP         158
APRN         76
           ... 
ARPN          1
ARNP,MS       1
ARNP,CNM      1
ARNP,BC       1
C,E,FNP       1
Name: count, Length: 154, dtype: int64

Provider_Credentials column

1. dropping all spaces
2. make credentials alphabetically ordered with each cell
3. order alphabetically


mapping function for missing subspecialties

In [13]:
pd.crosstab(
    nps["Provider_Credentials"],
    nps["NP_Type"],
    normalize="index" 
).round(2)
    

NP_Type,Acute Care NP,Adult/Gero NP,Community Health NP,Critical Care NP,Family NP,"NP, No Subspecialty Noted",Neonatal Critical Care NP,Neonatal NP,OBGYN NP,Occupational Health NP,Pediatrics Critical Care NP,Pediatrics NP,Primary Care NP,Psych/Mental Health NP,School NP,Womens Health NP
Provider_Credentials,,,,,,,,,,,,,,,,
"A,APN,C",0.00,0.00,0.0,0.0,0.00,1.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00
"A,C,FNP",0.17,0.00,0.0,0.0,0.67,0.17,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00
"A,C,GNP",0.00,0.57,0.0,0.0,0.00,0.29,0.0,0.0,0.00,0.0,0.0,0.0,0.14,0.0,0.0,0.00
"A,CPNP",0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,1.0,0.00,0.0,0.0,0.00
"A,G,MSN",0.00,1.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNP,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,1.00
WHCNP,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.50,0.0,0.0,0.0,0.00,0.0,0.0,0.50
WHNP,0.00,0.00,0.0,0.0,0.02,0.06,0.0,0.0,0.09,0.0,0.0,0.0,0.00,0.0,0.0,0.83
